<center>
    <table>
        <tr>
            <th> <img src="figures/ufrr_logo.png" width=75px> </th>
            <th> <img src="figures/dcc.jpg" width=100px> </th>
        </tr>
    <table>
    <font size=4>
    Universidade Federal de Roriama
    <br>
    Departamento de Ciência da Computação
    <br>
    Professor: Filipe Dwan Pereira
    <br>
    Código da disciplina: DCC305
    <br>
    Período: 2019.2
    </font>
</center>

## Disclaimer

Esta aula é uma adaptação do capítulo 3 do livro:

* Phillips, Dusty. Python 3 Object-oriented Programming - Unleash the power of Python 3 objects. "Packt Publishing", 2015. Second Edition.

Nesta aula aprenderemos:

* O básico sobre herança
* Usar herança a partir de built-ins
* Herança Múltipla;
* Polimorfismos e duck typing;

## Herança Básica

* Tecnicamente, todas as classes que usamos já usam herança, uma vez que elas herdam da classe *object*;
    * Os comportamentos fornecidos por essa classe são aqueles com underscore duplo;
* A classe object permite que o python trate todos os objetos da mesma maneira, isto é, como um objeto;
* Até agora ao clicar nossas classes não explicitamos que herdamos de object, mas podemos, conforme segue:

In [ ]:
class MySubClass(object):
    pass

* Isso é **herança**;
* Chamamos de superclasse ou classe pai a classe a qual herdamos e de subclasse ou classe filha a classe que herda.
    * No caso acima, object é a superclasse e MySubClasse a subclasse;
* Para realizar a herança em python basta colocar o nome da superclasse a qual se quer herdar dentro dos parênteses da definidação da subclasse;

* O uso mais simples da herança é adicionar funcionalidade a uma classe existente.
* Para ilustrar, veja um simples exemplo de uma classe base que gerencia contados:

In [10]:
class Contact:
    all_contacts = []
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

* Neste exemplo usamos o que chamamos de variável de classe;
    * A lista *all_contacts* é uma variável de classe porque é compartilhada por todos objetos da classe *Contact*.
* Agora imagine que existem contatos de fornecedores que fazemos pedidos;
    * Poderíamos pensar em adicionar um método **order** na classe Contact, mas isso faria que contatos familiares também tivessem esse método;
    * Vamos criar uma classe Fornecedores (*Supplier* em inglês) que além de herdar de Contact, também possui o método *order*:

In [11]:
class Supplier(Contact):
    def order(self, pedido):
        print("Pedido para '{}': "
            "'{}'".format(self.name, pedido))

In [12]:
c1 = Contact('Filipe', 'filipe@gmail.com')
f1 = Supplier('Amazon', 'amazon@gmail.com')

* Note que o objeto f1 possui nome e email pois herda da classe Contact:

In [13]:
print(c1.name, c1.email)
print(f1.name, f1.email)

Filipe filipe@gmail.com
Amazon amazon@gmail.com


* Perceba ainda que apenas o objeto f1 é capaz de efetuar pedidos:

In [14]:
f1.order('Preciso de 30 caixas de canetas')

Pedido para 'Amazon': 'Preciso de 30 caixas de canetas'


In [15]:
c1.order('Preciso de 30 caixas de canetas')

AttributeError: 'Contact' object has no attribute 'order'

* Agora perceba como o atributo *all_contacts* é compartilhado por ambos objetos:

In [16]:
c1.all_contacts

[<__main__.Contact at 0x7fd362fc1da0>, <__main__.Supplier at 0x7fd362fc1710>]

In [17]:
f1.all_contacts

[<__main__.Contact at 0x7fd362fc1da0>, <__main__.Supplier at 0x7fd362fc1710>]

## Herdando de built-ins

* Um uso interessante desse tipo de herança é adicionar funcionalidade aos recursos built-ins do python;
* Para exemplo, podemos criar uma classe *ContactList* que adiciona um método para fazer busca de contatos em uma lista padrão do python;
    * Para tanto, herdaremos de *list*:
* Após isso, iremos usar a ContectList ao invés de uma lista padrão para iniciazar nossa variável de classe *all_contacts* (linha 10 do código abaixo):

In [18]:
class ContactList(list):    
    def search(self, name):
        matching_contacts = []
        for contact in self:
            if name in contact.name:
                matching_contacts.append(contact)
        return matching_contacts

class Contact():
    all_contacts = ContactList()
    
    def __init__(self, name, email):
        self.name = name
        self.email = email
        Contact.all_contacts.append(self)

* Para testar nosso exemplo, vamos criar três contatos:

In [19]:
c1 = Contact('Fulano da Silva', 'fulano1@dominio.com')
c2 = Contact('Fulano Sousa', 'fulano2@dominio.com')
c3 = Contact('Ciclano Pereira', 'ciclano3@dominio.com')

* Agora vamos fazer uma busca por contatos com nome *Fulano*;

In [20]:
Contact.all_contacts.search('Fulano')

[<__main__.Contact at 0x7fd36333c828>, <__main__.Contact at 0x7fd36333cc50>]

* Bem, existem dois objetos com esse nome;
    * Vamos apresentá-los:

In [21]:
[c.name for c in Contact.all_contacts.search('Fulano')]

['Fulano da Silva', 'Fulano Sousa']

* Note que a sintaxe built-in [ ] é equivalante a *list()*:

In [22]:
[] == list()

True

* Assim sendo, quando fazemos *variavel = [ ]*, estamos instanciando um objeto da classe *list*;
* Ainda, a classe *list* herda de object, como podemos ver abaixo:

In [23]:
isinstance([], object)

True

* Como um segundo exemplo, vamos herdar agora da classe *dict*;
* Iremos adicionar uma funcionalidade para verificar qual é a chave mais longa do dicionário:

In [24]:
class LongNameDict(dict):
    def longest_key(self):
        longest = None
        for key in self.keys():
            if not longest or len(key) > len(longest):
                longest = key
        return longest

In [25]:
longkeys = LongNameDict()
longkeys['hello'] = 1
longkeys['longest yet'] = 5
longkeys['hello2'] = 'world'
longkeys.longest_key()

'longest yet'

## Sobrescrevendo métodos

* Além de adicionar funcionalidades ao utilizar herença, podemos também modificá-las;
    * Isto é, podemos herdar comportamentos e modificá-los;
* Por exemplo, imagine que queremos adicionar o atributo phone_number nos contatos dos nossos amigos mais próximos;
    * Para tanto, podemos criar uma classe Friend que herda de Contact e sobrescreve o construtor:

In [26]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        self.name = name
        self.email = email
        self.phone = phone

* Observe 2 problemas com o código acima:
    1. Repetimos código o que pode tornar o processo de manutenção do sistema ruim;
    2. Esquecemos de adicionar o contado do amigo na variável de classe *all_contacts*;
* O que nós realmente precisamos é executar o \_\_init\_\_ da classe Contact e apenas adicionar o telefone do contato;
* Para tanto, podemos usar a palavra chave **super**, que nos permite invocar métodos da superclasse diretamente:

In [27]:
class Friend(Contact):
    def __init__(self, name, email, phone):
        super().__init__(name, email)
        self.phone = phone

* Vamos criar um amigo, apenas para testar a classe:

In [28]:
friend1 = Friend('Beltrano Ferreira', 'beltrano@dominio.com', '2121-3131')

In [29]:
friend1.phone

'2121-3131'

## Herança Múltipla

* A herança múltipla é um assunto delicado.
    * Em teoria, é bem simples: uma subclasse que herda diretamente funcionalidades de mais uma superclasse;
    * Na prática, usa técnica não é tão usual e muitos experts não recomendam essa prática;
* A maneira mais fácil de realizar herança é chamda de **mixin**;
    * Mixin é geralmente uma superclasse que só existe para fornecer a herança;
    * Por exemplo, vamos adicionar funcionalidades a nossa classe *Contact* para poder enviar e-mail para o *self.email*;

In [30]:
class MailSender:
    def send_mail(self, message):
        print('Enviando e-mail para ' + self.email)
        # logica de envio de e-mail

* Podemos então criar uma classe que herda de Cotact e MailSender:

In [31]:
class EmailableContact(Contact, MailSender):
    pass

* Agora vamos testar a nossa classe híbrida:

In [32]:
e = EmailableContact("John Smith", "jsmith@example.net")

In [35]:
e.send_mail("Hello, test e-mail here")

Enviando e-mail para jsmith@example.net


* Observe que o objeto da classe *EmailableContact* também está sendo adicionado na lista *all_contacts*:

In [34]:
Contact.all_contacts

### Nota sobre envio de email em python

* Caso queira realmente enviar um e-mail, segue abaixo um breve tutorial onde o remetente tem uma conta do gmail.
    * Primeiro, acesse este [link](https://myaccount.google.com/lesssecureapps?pli=1) e dê permissão para aplicações menos seguras;
    * A seguir, use o seguinte código:

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

email = 'seu_email@gmail.com'
senha = input('Entre com a senha:')
receptor = 'seu_email@gmail.com'
assunto = 'Ufrr python - POO'
conteudo = 'Aula de POO - Testando o envio de um email'

msg = MIMEMultipart()
msg['From'] = email
msg['To'] = receptor
msg['Subject'] = assunto
msg.attach(MIMEText(conteudo, 'plain'))

servidor = smtplib.SMTP('smtp.gmail.com', 587)
servidor.starttls()
servidor.login(email, senha)
text = msg.as_string()
servidor.sendmail(email, receptor, text)
servidor.quit()

* **Créditos**: o tutorial acima foi feito com base nesta [videoaula](https://www.youtube.com/watch?v=YPiHBtddefI) do canal PyTutorials;
* Para um tutorial um pouco mais completo, acesso este [link](https://realpython.com/python-send-email/) do site real python;

### Um pouco mais sobre herança múltipla

* Note que poderíamos ter usado composição ao invés de herança na classe EmailableContact:
    * A classe EmailableContact poderia ter um objeto MailSender ao invés de herdar dele, o que faz muito mais sentido;
* Herança Múltipla pode se tornar bastante confuso quando precisamos invocar métodos da superclasse, uma vez que existem múltiplas superclasses;
    * Para explorar um pouco mais sobre esse assunto, vamos criar uma classe que que modela objetos detentores de endereço:

In [36]:
class AddressHolder:
    def __init__(self, street, city, state, code):
        self.street = street
        self.city = city
        self.state = state
        self.code = code

* Vamos usar a classe acima com pai da nossa classe Friend;
* Entretanto, percebam que a classe Friend já herda de Contact e que usamos a palavra chave super para modificar um comportamento.
    * Com hernaça múltipla, ao usar por exemplo, *super().\_\_init\_\_()* estamos nos referindo a qual das superclasses?
    * Uma opção para tentar contornar esse problema é especificar a super classe e passar explicitamento o self como parâmetro no \_\_init\_\_() da superclasse, conforme segue:

In [38]:
class Friend(Contact, AddressHolder):
    def __init__(self, name, email, phone, street, city, state, code):
        Contact.__init__(self, name, email)
        AddressHolder.__init__(self, street, city, state, code)
        self.phone = phone

* O código acima funciona, mas é perigoso. 
* Veja o seguinte diagrama de classes:

<img src="figures/class_diagram_mult_inheritance.png"/>

* Observe que:
    * O método \_\_init\_\_() da classe Friends invoca o  \_\_init\_\_() de Contact, que implicitamente invoca o \_\_init\_\_() da classe Object;
    * Subsequentemente, o método \_\_init\_\_() da classe Friends invoca o  \_\_init\_\_() de AddressHolder, que também implicitamente invoca o \_\_init\_\_() da classe Object;
    * Assim, o \_\_init\_\_() da classe Object é chamado duas vezes;
    * Para este cenário não há problema algum, mas imagine que no lugar de Object está a classe Database e no \_\_init\_\_() do Database temos uma conexão com o banco de dados;
        * Faríamos a conexão duas vezes para cada requisição, o que pode ser desastroso;

* Vejamos um segundo exemplo hipotético que ilustra esse problema mais claramente:
    * Aqui temos uma classe base que possui um método chamado *call_me*;
    * As subclasses LeftSubClass e RightSubClass herdam diretamente de BaseClass e sobreescrevem o método *call_me*;
    * A classe Subclass herda de LeftSubClass e RightSubClass e também sobreescreve *call_me*;
*  Isso é chamado de herança de diamante (diamond inheritance) devido à forma do diamante do diagrama de classes:

<img src="figures/class_diagram_diamond.png"/>

* Vamos converter esse diagrama em código:

In [39]:
class BaseClass:
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1
        
class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self):
        BaseClass.call_me(self)#linha importante
        print("Calling method on Left Subclass")
        self.num_left_calls += 1

class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self):
        BaseClass.call_me(self)#linha importante
        print("Calling method on Right Subclass")
        self.num_right_calls += 1
        
class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self):
        LeftSubclass.call_me(self)#linha importante
        RightSubclass.call_me(self)#linha importante
        print("Calling method on Subclass")
        self.num_sub_calls += 1

* Vamos testar a nossa subclasse:

In [41]:
s = Subclass()
s.call_me()
print(s.num_sub_calls)
print(s.num_left_calls)
print(s.num_right_calls)
print(s.num_base_calls)

Calling method on Base Class
Calling method on Left Subclass
Calling method on Base Class
Calling method on Right Subclass
Calling method on Subclass
1
1
1
2


* Aqui podemos ver claramente a classe BaseClass sendo chamada duas vezes;
* Apenas como mais uma demostração dos problemas que isso pode causar, imagine que esse método está realizando um depósito em um banco;
*  O que você devemos ter em mente com herança múltipla é que só queremos chamar o método "filho" na hierarquia de classes, não o método "pai";

* Podemos resolver isso usando o nosso **super**, que se encarregará de chamar o método da superclasse apenas uma vez;
* Vamos reescrever nossa classe agora utilizando o super:

In [42]:
class BaseClass:
    num_base_calls = 0
    def call_me(self):
        print("Calling method on Base Class")
        self.num_base_calls += 1
        
class LeftSubclass(BaseClass):
    num_left_calls = 0
    def call_me(self):
        super().call_me()#linha importante
        print("Calling method on Left Subclass")
        self.num_left_calls += 1

class RightSubclass(BaseClass):
    num_right_calls = 0
    def call_me(self):
        super().call_me()#linha importante
        print("Calling method on Right Subclass")
        self.num_right_calls += 1
        
class Subclass(LeftSubclass, RightSubclass):
    num_sub_calls = 0
    def call_me(self):
        super().call_me()#linha importante
        print("Calling method on Subclass")
        self.num_sub_calls += 1

* Agora vamos novamente o nosso teste e perceba que a classe base só será chamada uma vez:

In [43]:
s = Subclass()
s.call_me()
print(s.num_sub_calls)
print(s.num_left_calls)
print(s.num_right_calls)
print(s.num_base_calls)

Calling method on Base Class
Calling method on Right Subclass
Calling method on Left Subclass
Calling method on Subclass
1
1
1
1


Vale ressaltar como funcionou esse processo para que a classe base seja chamada apenas uma vez:
* Primeiro foi invocado *call_me* da SubClass que chamou *super().call_me()*, que se refere a *LeftSubClass.call_me()*. 
* *LeftSubClass.call_me()*, por sua vez, chama *super().call_me()*, mas nesse caso, *super()* está se referindo a *RightSubclass.call_me()*;
    * Assim sendo, o *super* na verdade não está sendo chamado na classe *LeftSubClass*;
    * Ao invés disso, ele é chamado na *RightSubClass*;
    * A *RightSubclass* que chama o *call_me()* da *BaseClass*;